In [ ]:
https://shankarmsy.github.io/stories/gbrt-sklearn.html
https://shankarmsy.github.io/stories/gbrt-sklearn.html    

In [1]:
%matplotlib inline 
import matplotlib.pylab as plt 
import numpy as np 
from scipy import sparse
from sklearn.datasets import make_classification, make_blobs, load_boston, fetch_california_housing
from sklearn.decomposition import PCA 

from sklearn.model_selection import ShuffleSplit, train_test_split, learning_curve, GridSearchCV 
from sklearn import metrics 

from sklearn.linear_model import LinearRegression 
from sklearn.ensemble import GradientBoostingRegressor
from pprint import pprint 
import pandas as pd 
from pandas.tools.plotting import scatter_matrix 
import urllib 
import requests 
import zipfile 
import seaborn 

In this notebook, we use [loan dataset]("https://resources.lendingclub.com/LoanStats3c.csv.zip"). Note that all member ids, url field are missing. More [details]("https://resources.lendingclub.com/LCDataDictionary.xlsx") 

In [2]:
loan=pd.read_csv("C:/Dataset/loanStats3c.csv", skiprows=1, parse_dates=True, index_col='id', low_memory=False) 
loanbk=loan.copy()

In [3]:
#For simplicity, remove columns wtith all missing values in the dataset. 
#loan = loan.dropna(axis=0) 
loan= loan.dropna(axis=1, how='all')
loan.isnull().sum()

loan_amnt                                          2
funded_amnt                                        2
funded_amnt_inv                                    2
term                                               2
int_rate                                           2
installment                                        2
grade                                              2
sub_grade                                          2
emp_title                                      13238
emp_length                                         2
home_ownership                                     2
annual_inc                                         2
verification_status                                2
issue_d                                            2
loan_status                                        2
pymnt_plan                                         2
desc                                          220352
purpose                                            2
title                                         

In [4]:
# What is a percentage of missing values
232888/235631

0.9883589171204128

In [5]:
#Let decide threshold 0.97 andfilter out all columns with missing values more than 97%
loan=loan.dropna(thresh=len(loan) * 0.97, axis=1)

In [6]:
# Drop fields s.t it contains unique values
loan=loan.drop([ 'grade', 'sub_grade',  'issue_d', 'pymnt_plan', 'title', 'initial_list_status', 'last_pymnt_d',\
        'last_pymnt_amnt', 'last_credit_pull_d', 'policy_code','emp_length', 'addr_state','zip_code','application_type'], axis=1) 
loan.head(4)

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag
id,,,,,,,,,,,,,,,,,,,,,
NaN,10400.0,10400.0,10400.0,36 months,6.99%,321.08,MORTGAGE,58000.0,Not Verified,Charged Off,...,14.3,0.0,0.0,179407.0,15030.0,13000.0,11325.0,N,Cash,N
NaN,15000.0,15000.0,15000.0,60 months,12.39%,336.64,RENT,78000.0,Source Verified,Fully Paid,...,0.0,0.0,0.0,196500.0,149140.0,10000.0,12000.0,N,Cash,N
NaN,21425.0,21425.0,21425.0,60 months,15.59%,516.36,RENT,63800.0,Source Verified,Fully Paid,...,100.0,0.0,0.0,57073.0,42315.0,15000.0,35573.0,N,Cash,N
NaN,12800.0,12800.0,12800.0,60 months,17.14%,319.08,MORTGAGE,125000.0,Verified,Current,...,100.0,0.0,0.0,368700.0,18007.0,4400.0,18000.0,N,Cash,N


## Approach 1: 

In [7]:
# Get rid of non-numeric values throughout the DataFrame: 
for col in loan.columns.values: 
    loan[col] = loan[col].replace('[^0-9]+.-', '', regex=True) 
loan.head(2) 

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag
id,,,,,,,,,,,,,,,,,,,,,
NaN,10400.0,10400.0,10400.0,36 months,6.99%,321.08,MORTGAGE,58000.0,Not Verified,Charged Off,...,14.3,0.0,0.0,179407.0,15030.0,13000.0,11325.0,N,Cash,N
NaN,15000.0,15000.0,15000.0,60 months,12.39%,336.64,RENT,78000.0,Source Verified,Fully Paid,...,0.0,0.0,0.0,196500.0,149140.0,10000.0,12000.0,N,Cash,N


In [8]:
#Remove symbol from the interest rate & revolving utilization 
loan.int_rate=loan.int_rate.str.split('%',1).str[0]
loan.revol_util=loan.revol_util.str.split('%',1).str[0] 
#Remove "months" from the loan period 
loan.term=loan.term.str.split(' ',2).str[1]
loan.head(2) 

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag
id,,,,,,,,,,,,,,,,,,,,,
NaN,10400.0,10400.0,10400.0,36,6.99,321.08,MORTGAGE,58000.0,Not Verified,Charged Off,...,14.3,0.0,0.0,179407.0,15030.0,13000.0,11325.0,N,Cash,N
NaN,15000.0,15000.0,15000.0,60,12.39,336.64,RENT,78000.0,Source Verified,Fully Paid,...,0.0,0.0,0.0,196500.0,149140.0,10000.0,12000.0,N,Cash,N


### Note:
When we use pandas.select_dtypes to filter numeric and category fields, it does not work properly when numeric fields have missing values. The reason is that pandas may transform these columns into **object** rather numerics. As a result, these fields are treated as textual feature. We need to assign/ force pandas to convert missing/text values into **NA** while remaining values as numeric. Eg:      
> **df[['col2','col3']] = df[['col2','col3']].apply(pd.to_numeric)**

In [13]:
# split data into numeric and categorical sets
def split_categories(df):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    df_n = df.select_dtypes(include=numerics).copy()
    df_c = df.select_dtypes(exclude=numerics).copy()
    
    return df_n, df_c
df_n,df_c= split_categories(loan)
df_c.head()

,term,int_rate,home_ownership,verification_status,loan_status,purpose,earliest_cr_line,revol_util,hardship_flag,disbursement_method,debt_settlement_flag
id,,,,,,,,,,,
NaN,36,6.99,MORTGAGE,Not Verified,Charged Off,credit_card,1989,31.6,N,Cash,N
NaN,60,12.39,RENT,Source Verified,Fully Paid,debt_consolidation,1994,29,N,Cash,N
NaN,60,15.59,RENT,Source Verified,Fully Paid,credit_card,2003,76.2,N,Cash,N
NaN,60,17.14,MORTGAGE,Verified,Current,car,2000,100.9,N,Cash,N
NaN,36,13.66,RENT,Source Verified,Fully Paid,debt_consolidation,1992,59.4,N,Cash,N


We observe that four **numeric features** being considered as textual attributes such as **term**,**int_rate**, **earliest_cr_line**, **revol_util** due to pandas read them as object. One reason is that pandas daframe detect corrupted data (blank) that results in object type. Let see

In [14]:
df_c[['term','int_rate', 'earliest_cr_line', 'revol_util']].isnull().sum()

term                  2
int_rate              2
earliest_cr_line      2
revol_util          127
dtype: int64

We force convert these features into numeric so that missing values are represented as NA

In [15]:
fields=['term','int_rate', 'earliest_cr_line', 'revol_util']
df_c[fields]=df_c[fields].apply(pd.to_numeric,errors='coerce',axis=1)
df_c.tail()

,term,int_rate,home_ownership,verification_status,loan_status,purpose,earliest_cr_line,revol_util,hardship_flag,disbursement_method,debt_settlement_flag
id,,,,,,,,,,,
NaN,36.0,7.90,OWN,Verified,Fully Paid,credit_card,2003.0,21.5,N,Cash,N
NaN,60.0,16.99,MORTGAGE,Verified,Current,debt_consolidation,2001.0,51.1,N,Cash,N
NaN,36.0,19.20,MORTGAGE,Verified,Fully Paid,other,2000.0,70.8,N,Cash,N
Total amount funded in policy code 1: 3503840175,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total amount funded in policy code 2: 873652739,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df_c.info()

<class 'pandas.core.frame.DataFrame'>
Index: 235631 entries, nan to Total amount funded in policy code 2: 873652739
Data columns (total 11 columns):
term                    235629 non-null float64
int_rate                235629 non-null float64
home_ownership          235629 non-null object
verification_status     235629 non-null object
loan_status             235629 non-null object
purpose                 235629 non-null object
earliest_cr_line        235629 non-null float64
revol_util              235504 non-null float64
hardship_flag           235629 non-null object
disbursement_method     235629 non-null object
debt_settlement_flag    235629 non-null object
dtypes: float64(4), object(7)
memory usage: 21.6+ MB


In [17]:
# Our strategy is to use the most frequent values for term, earliest_cr_line and median for int_rate, revol_util
df_c['term'].fillna(df_c['term'].value_counts().index[0], inplace=True)
df_c['earliest_cr_line'].fillna(df_c['earliest_cr_line'].value_counts().index[0], inplace=True)
df_c['int_rate'].fillna(df_c['int_rate'].median(), inplace=True)
df_c['revol_util'].fillna(df_c['revol_util'].median(), inplace=True)

 Fill remaining missing values with the most frequent values

In [19]:
fields = ['home_ownership','verification_status','loan_status','purpose','hardship_flag','disbursement_method','debt_settlement_flag']
df_c[fields]=df_c[fields].apply(lambda x:x.fillna(x.value_counts().index[0]))

In [20]:
df_c.isnull().sum()

term                    0
int_rate                0
home_ownership          0
verification_status     0
loan_status             0
purpose                 0
earliest_cr_line        0
revol_util              0
hardship_flag           0
disbursement_method     0
debt_settlement_flag    0
dtype: int64

In [21]:
df_n.head(3)

,loan_amnt,funded_amnt,funded_amnt_inv,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
id,,,,,,,,,,,,,,,,,,,,,
NaN,10400.0,10400.0,10400.0,321.08,58000.0,14.92,0.0,2.0,17.0,0.0,...,0.0,4.0,83.3,14.3,0.0,0.0,179407.0,15030.0,13000.0,11325.0
NaN,15000.0,15000.0,15000.0,336.64,78000.0,12.03,0.0,0.0,6.0,0.0,...,0.0,4.0,100.0,0.0,0.0,0.0,196500.0,149140.0,10000.0,12000.0
NaN,21425.0,21425.0,21425.0,516.36,63800.0,18.49,0.0,0.0,10.0,0.0,...,0.0,2.0,91.4,100.0,0.0,0.0,57073.0,42315.0,15000.0,35573.0


In [22]:
df_n=df_n.apply(lambda x:x.fillna(round(x.mean())))

In [23]:
loan = pd.concat([df_n,df_c],axis=1)
loan.head(3)

,loan_amnt,funded_amnt,funded_amnt_inv,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,int_rate,home_ownership,verification_status,loan_status,purpose,earliest_cr_line,revol_util,hardship_flag,disbursement_method,debt_settlement_flag
id,,,,,,,,,,,,,,,,,,,,,
NaN,10400.0,10400.0,10400.0,321.08,58000.0,14.92,0.0,2.0,17.0,0.0,...,6.99,MORTGAGE,Not Verified,Charged Off,credit_card,1989.0,31.6,N,Cash,N
NaN,15000.0,15000.0,15000.0,336.64,78000.0,12.03,0.0,0.0,6.0,0.0,...,12.39,RENT,Source Verified,Fully Paid,debt_consolidation,1994.0,29.0,N,Cash,N
NaN,21425.0,21425.0,21425.0,516.36,63800.0,18.49,0.0,0.0,10.0,0.0,...,15.59,RENT,Source Verified,Fully Paid,credit_card,2003.0,76.2,N,Cash,N


### Overview of textual features

In [24]:
loan.home_ownership.value_counts()

MORTGAGE    119939
RENT         92684
OWN          23007
ANY              1
Name: home_ownership, dtype: int64

In [25]:
loan.verification_status.value_counts()

Source Verified    97743
Not Verified       70659
Verified           67229
Name: verification_status, dtype: int64

In [26]:
loan.loan_status.value_counts()

Fully Paid            159890
Charged Off            37225
Current                35480
Late (31-120 days)      1662
In Grace Period          949
Late (16-30 days)        401
Default                   24
Name: loan_status, dtype: int64

In [27]:
loan.purpose.value_counts()

debt_consolidation    143008
credit_card            55522
home_improvement       13045
other                  10371
major_purchase          3858
medical                 2331
small_business          2277
car                     1832
moving                  1328
vacation                1178
house                    750
renewable_energy         123
wedding                    8
Name: purpose, dtype: int64

In [28]:
loan.hardship_flag.value_counts()

N    235465
Y       166
Name: hardship_flag, dtype: int64

In [29]:
loan.disbursement_method.value_counts()

Cash    235631
Name: disbursement_method, dtype: int64

In [30]:
loan.debt_settlement_flag.value_counts()

N    232888
Y      2743
Name: debt_settlement_flag, dtype: int64

 Lend club [term]("https://help.lendingclub.com/hc/en-us/articles/216127747-What-is-the-difference-between-a-loan-that-is-in-default-and-a-loan-that-has-been-charged-off-") defines    
Loans in **Default** are loans for which borrowers have failed to make payments for an extended period of time.  In general, a note enters Default status when it is 121+ days past due.
 
A loan becomes **Charged Off** when there is no longer a reasonable expectation of further payments.  Charge Off typically occurs when a loan is 150 days past due (i.e. 30 days after the Default status is reached) and there is no reasonable expectation of sufficient payment to prevent the charge off.  In certain circumstances, loans may be charged off at an earlier or later date.  

## What you are thinking
In a first attempt, experiment could not finish during 4 gio. Since we observe values of many fields with small ranges. Can you try with a better computational power, or optimize data type

In [33]:
loan.head()

,loan_amnt,funded_amnt,funded_amnt_inv,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,int_rate,home_ownership,verification_status,loan_status,purpose,earliest_cr_line,revol_util,hardship_flag,disbursement_method,debt_settlement_flag
id,,,,,,,,,,,,,,,,,,,,,
NaN,10400.0,10400.0,10400.0,321.08,58000.0,14.92,0.0,2.0,17.0,0.0,...,6.99,MORTGAGE,Not Verified,Charged Off,credit_card,1989.0,31.6,N,Cash,N
NaN,15000.0,15000.0,15000.0,336.64,78000.0,12.03,0.0,0.0,6.0,0.0,...,12.39,RENT,Source Verified,Fully Paid,debt_consolidation,1994.0,29.0,N,Cash,N
NaN,21425.0,21425.0,21425.0,516.36,63800.0,18.49,0.0,0.0,10.0,0.0,...,15.59,RENT,Source Verified,Fully Paid,credit_card,2003.0,76.2,N,Cash,N
NaN,12800.0,12800.0,12800.0,319.08,125000.0,8.31,1.0,0.0,8.0,0.0,...,17.14,MORTGAGE,Verified,Current,car,2000.0,100.9,N,Cash,N
NaN,9600.0,9600.0,9600.0,326.53,69000.0,25.81,0.0,0.0,12.0,0.0,...,13.66,RENT,Source Verified,Fully Paid,debt_consolidation,1992.0,59.4,N,Cash,N


In [78]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
fields = ['home_ownership','verification_status','loan_status','purpose','hardship_flag','disbursement_method','debt_settlement_flag']
loan[fields] = loan[fields].apply(lambda x:le.fit_transform(x))
loan.head()

,loan_amnt,funded_amnt,funded_amnt_inv,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,int_rate,home_ownership,verification_status,loan_status,purpose,earliest_cr_line,revol_util,hardship_flag,disbursement_method,debt_settlement_flag
id,,,,,,,,,,,,,,,,,,,,,
NaN,10400.0,10400.0,10400.0,321.08,58000.0,14.92,0.0,2.0,17.0,0.0,...,6.99,1,0,0,1,1989.0,31.6,0,0,0
NaN,15000.0,15000.0,15000.0,336.64,78000.0,12.03,0.0,0.0,6.0,0.0,...,12.39,3,1,3,2,1994.0,29.0,0,0,0
NaN,21425.0,21425.0,21425.0,516.36,63800.0,18.49,0.0,0.0,10.0,0.0,...,15.59,3,1,3,1,2003.0,76.2,0,0,0
NaN,12800.0,12800.0,12800.0,319.08,125000.0,8.31,1.0,0.0,8.0,0.0,...,17.14,1,2,1,0,2000.0,100.9,0,0,0
NaN,9600.0,9600.0,9600.0,326.53,69000.0,25.81,0.0,0.0,12.0,0.0,...,13.66,3,1,3,2,1992.0,59.4,0,0,0


In [11]:
#Let's change the Income Verified column, which currently has textual labels to numeric.
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
loan.home_ownership=le.fit_transform(loan.home_ownership.values)
loan.loan_status=le.fit_transform(loan.loan_status.values)
loan.purpose=le.fit_transform(loan.purpose.values)

#Finally let's be sure we convert all fields to numeric
loan=loan.convert_objects(convert_numeric=True)

loan.head(2)

AttributeError: 'DataFrame' object has no attribute 'is_inc_v'

In [83]:
def GradientBooster(param_grid, n_jobs): 
    estimator = GradientBoostingRegressor()
    cv = ShuffleSplit(X_train.shape[0],  test_size=0.2)
    classifier = GridSearchCV(estimator=estimator, cv=cv, param_grid=param_grid, n_jobs=n_jobs)
    classifier.fit(X_train, y_train)
    print("Best Estimator learned through GridSearch")
    print(classifier.best_estimator_)
    return cv, classifier.best_estimator_ 

In [79]:
#OK great, let's now get our X and y. We know that interest rate is y. 
y=loan.int_rate.values 
#Let's remove y from the df so we can get X 
del loan['int_rate'] 
X=loan.values #Now, the train test split 
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [ ]:
param_grid={'n_estimators':[100],#,500,1000],
            'learning_rate': [0.1,0.05,0.02],# 0.01],
            'max_depth':[4,6],
            'min_samples_leaf':[3,5,9,17], 
            'max_features':[1.0,0.3,0.1] }
n_jobs=4 
#Let's fit GBRT to the digits training dataset by calling the function we just created. 
cv,best_est=GradientBooster(param_grid, n_jobs) 

In [ ]:
print("Best Estimator Parameters")
print("---------------------------")
print("n_estimators: %d" %best_est.n_estimators)
print("max_depth: %d" %best_est.max_depth)
print("Learning Rate: %.1f" %best_est.learning_rate)
print("min_samples_leaf: %d" %best_est.min_samples_leaf)
print("max_features: %.1f" %best_est.max_features)
print
print("Train R-squared: %.2f" %best_est.score(X_train,y_train))
#The training R-Squared is almost 1.0 which indicates we can understand 99% of the variance in the data as well as 
#there's a chance we might overfit. Let's see with the learning curves below. 

In [ ]:
title = "Learning Curves (Gradient Boosted Regression Trees)" 
estimator = GradientBoostingRegressor(n_estimators=best_est.n_estimators, max_depth=best_est.max_depth, learning_rate=best_est.learning_rate, min_samples_leaf=best_est.min_samples_leaf, max_features=best_est.max_features) 
plot_learning_curve(estimator, title, X_train, y_train, cv=cv, n_jobs=n_jobs)
plt.show() 

In [ ]:
#the typical recommended values of Max depth is 4 to 6, so lets leave it at 4. Let's try increasing the min samples leaf
#parameter, this basically enforces a lower bound on the number of samples in any given leaf. 
min_samples_leaf=9
title = "Learning Curves (Gradient Boosted Regression Trees), min_samples_leaf=9" 
estimator = GradientBoostingRegressor(n_estimators=best_est.n_estimators, max_depth=best_est.max_depth, learning_rate=best_est.learning_rate, min_samples_leaf=min_samples_leaf, max_features=best_est.max_features)
plot_learning_curve(estimator, title, X_train, y_train, cv=cv, n_jobs=n_jobs)
plt.show() 

In [ ]:
#Let's try reducing the max features parameter. This enforces an upper bound of the maximum number of features to use #for training. It's supposed to work well when n_features>30. We'll also remove min samples leaf for this run.
max_features=0.5 
title = "Learning Curves (Gradient Boosted Regression Trees), max_features=50%"
estimator = GradientBoostingRegressor(n_estimators=best_est.n_estimators, max_depth=best_est.max_depth, learning_rate=best_est.learning_rate, max_features=max_features)
plot_learning_curve(estimator, title, X_train, y_train, cv=cv, n_jobs=n_jobs)
plt.show()
#Nope that didn't quite improve the cv score either. What happens if we reduce learning rate? 

In [ ]:
#The lower the learning rate is the more the number of trees we need to train. This is because the rate at which we train
#is simply, well, reduced. 
learning_rate=.01 
n_estimators=1000 
title = "Learning Curves (Gradient Boosted Regression Trees), 1000 Trees at learning rate .01" 
estimator = GradientBoostingRegressor(n_estimators=n_estimators, max_depth=best_est.max_depth, learning_rate=learning_rate, min_samples_leaf=best_est.min_samples_leaf, max_features=best_est.max_features) 
plot_learning_curve(estimator, title, X_train, y_train, cv=cv, n_jobs=n_jobs) 
plt.show()
#Perhaps that improved it a tiny little bit. 

In [ ]:
#Before we try anything else, I would like to explore one of the beautiful advantages of growing trees. And that is to 
#capture feature importances. Now that we have a publicly available loan application collection (though anonymous), it makes 
#me really curious to see what impacts the interest rate for a loan application the most.
#Let's take a look 
#Calling fit on the estimator so we can look at feature_importances. 
estimator.fit(X_train, y_train) 
# Calculate the feature ranking - Top 10 
importances = estimator.feature_importances_ 
indices = np.argsort(importances)[::-1]
print("Lending Club Loan Data - Top 10 Important Features\n")
for f in range(10):
    print("%d. %s (%f)" % (f + 1, loan.columns[indices[f]], importances[indices[f]])) 
#Plot the feature importances of the forest 
indices=indices[:10] 
plt.figure() 
plt.title("Top 10 Feature importances")
plt.bar(range(10), importances[indices], color="r", align="center")
plt.xticks(range(10), loan.columns[indices], fontsize=14, rotation=45) 
plt.xlim([-1, 10])
plt.show() 
#Mean Feature Importance 
print "Mean Feature Importance %.6f" %np.mean(importances) 
#Interesting, the total amount of interest received to date is the top most influencer for getting a better interest rate. #Good for the lenders eh? Pay more interest, we'll give you a cut on the interest rate. Of course! 

In [ ]:
#Can we actually trim (like before) and get a better result? Perhaps not, but who's to stop us from trying.
title = "Learning Curves (Gradient Boosted Regression Trees) - Trimmed features to > 1% importance" 

#Dropping all parameters except n_estimators and learning_rate since we're going to trim the features anyway.
estimator = GradientBoostingRegressor(n_estimators=best_est.n_estimators, learning_rate=best_est.learning_rate)

#Calling fit on the estimator so we can transform the X matrices.
estimator.fit(X_train, y_train)

#Trimming feature matrices to include only those features that are more important than the mean of all importances.
X_train_trim=estimator.transform(X_train, threshold=.01)

#Trimming test as well in case we end up going with this model as final.
X_test_trim=estimator.transform(X_test, threshold=.01)

#Re-plotting Learning cruves.
plot_learning_curve(estimator, title, X_train, y_train, cv=cv, n_jobs=n_jobs)
plt.show()

#Nope, the curve looks like it overfits less, but look at the cv score, in all our fancy attempts it never really crossed
#that ~0.8 R-squared barrier. That tells me, we actually have a decent model at hand and also that a 0.9+ R-squared value is
#not always possible, atleast in real time. Let's wrap this up.



In [ ]:
estimator = GradientBoostingRegressor(n_estimators=best_est.n_estimators, max_depth=best_est.max_depth, learning_rate=best_est.learning_rate, min_samples_leaf=best_est.min_samples_leaf, max_features=best_est.max_features)
estimator.fit(X_train, y_train)
print("Final Estimator Parameters")
print("---------------------------")
print("n_estimators: %d" %best_est.n_estimators)
print( "max_depth: %d" %best_est.max_depth)
print("Learning Rate: %.1f" %best_est.learning_rate)
print("min_samples_leaf: %d" %best_est.min_samples_leaf)
print("max_features: %.1f" %best_est.max_features)
print 
print"Final Train R-squared: %.2f" %estimator.score(X_train, y_train))
print("Final Test R-squared: %.2f" %estimator.score(X_test, y_test)) 

There it is a final 0.83 R-squared value for the Test set. Compare this to the 0.72 we get with a LinearRegression model (look at the Linear Regression notebook in my repos) you'll know that we've done better. - See more at: 